In [4]:
%load_ext autoreload
%autoreload 2

import torch
import pprint
from pathlib import Path

In [5]:
from lerobot.datasets.lerobot_dataset import LeRobotDataset
from lerobot.policies.factory import make_policy
from lerobot.policies.smolvla.configuration_smolvla import SmolVLAConfig

c:\Users\19142\.conda\envs\my_robotics\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
from lerobot.policies.smolvla.modeling_smolvla import SmolVLAPolicy


In [7]:
repo_id = "lerobot/aloha_sim_transfer_cube_human"

dataset = LeRobotDataset(repo_id, episodes=[0])

print(f"Dataset loaded: {len(dataset)} frames")

pprint.pprint(dataset.meta.stats["action"])

Dataset loaded: 20000 frames
{'count': array([20000]),
 'max': array([ 0.04141748, -0.1043107 ,  1.24712646,  0.01227185, -0.2638447 ,
        0.13038836,  1.14149058,  0.33133987,  0.27918452,  1.2931459 ,
        0.25003886,  0.61205834,  1.22104871,  1.20042038]),
 'mean': array([-0.00756446, -0.62818439,  1.03128347, -0.04664316, -0.47211364,
       -0.07452702,  0.37389796, -0.03718753, -0.32611426,  0.8997206 ,
       -0.2137108 , -0.31840397, -0.2336096 ,  0.55194737]),
 'min': array([-0.07363108, -0.95873803,  0.68262148, -0.20248547, -0.83755356,
       -0.33747578,  0.15309308, -0.34054375, -1.04003906,  0.46939814,
       -1.44500995, -1.0154953 , -1.36217499,  0.14091808]),
 'std': array([0.01252788, 0.29573679, 0.1670151 , 0.04584395, 0.14833474,
       0.08762805, 0.30665162, 0.10599816, 0.2757198 , 0.18058076,
       0.26304034, 0.30707766, 0.53052784, 0.38380076])}


In [8]:
policy_cfg = SmolVLAConfig(
    n_action_steps=10,
    chunk_size=10,
    pretrained_path="lerobot/smolvla_base"
)

print("Building Policy...")
policy = make_policy(
    cfg=policy_cfg,
    ds_meta=dataset.meta
)

Building Policy...
Reducing the number of VLM layers to 16 ...


In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
policy.to(device)
print(f"device: {device}")

device: cuda


In [12]:
item = dataset[0]
# print(item.items())
batch = {k: v.unsqueeze(0).to(device) for k, v in item.items() if isinstance(v, torch.Tensor)}

print("Batch keys:", batch.keys())
print(f"Image shape: {batch['observation.images.top'].shape}")
print(f"Action shape: {batch['action'].shape}")

c:\Users\19142\.conda\envs\my_robotics\lib\site-packages\torchvision\io\_video_deprecation_warning.py:5: UserWarning: The video decoding and encoding capabilities of torchvision are deprecated from version 0.22 and will be removed in version 0.24. We recommend that you migrate to TorchCodec, where we'll consolidate the future decoding/encoding capabilities of PyTorch: https://github.com/pytorch/torchcodec
  warnings.warn(


Batch keys: dict_keys(['observation.images.top', 'observation.state', 'action', 'episode_index', 'frame_index', 'timestamp', 'next.done', 'index', 'task_index'])
Image shape: torch.Size([1, 3, 480, 640])
Action shape: torch.Size([1, 14])


In [13]:
policy.train()

loss, output_dict = policy.forward(batch)

print(f"Loss: {loss.item()}")
print("Output dict keys:", output_dict.keys())

KeyError: 'observation.language.tokens'